# <center>Text-TO-Text Task with LLMS </center>
- illustration prompt enginnering for text to text using Llama2
- Using the Bert_score and Rouge_score for the final evaltion 

## Importing required Libraries 

In [1]:
!pip install -q evaluate rouge_score bert_score
!pip install datasets
!pip install llama-cpp-python

In [13]:
import pandas as pd 
import numpy as np 
import json 
from collections import Counter
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from tqdm import tqdm
import torch 
import evaluate
from llama_cpp import Llama

## Practice Summarizing the score (Rouge,Bertscore)

#### Rouge_n_score

In [3]:
ai_generated_summary = "Colossal disappointment with constant glitches, defects, and a malfunctioning camera."
human_generated_summary = "Incredibly let down by the never-ending issues – glitches, defects, and a camera that just won't cooperate."

- The common words in aigenerated and humman_generated are 
* glitches,defects,and,a,camera
- Total length of ai_generated_summary is 10
- Total length of human_generated_summary 17
* recall of lcs(logestCommonSubsequence) human_generated = 5/17
* precision of lcs(logestCommonSubsequence) ai_generated = 5/10

In [6]:
r_lcs,p_lcs=5/17,5/10
rouge=(2*r_lcs*p_lcs)/(r_lcs+p_lcs)
rouge

0.37037037037037035

### Bert_score

- Bert score use one such Pre-Trained Embedding model (Bi-directional encoder representation from Transformers -Bert)
- Bert model take the semantic meaning of the words used in the models in a mathematical space 

In [9]:
bert_score=evaluate.load("bertscore")

In [12]:
bert_score.compute(
    predictions=[ai_generated_summary],
    references=[human_generated_summary],
    lang="en",
    rescale_with_baseline=True
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.6490632891654968],
 'recall': [0.48829349875450134],
 'f1': [0.5682036876678467],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.2)-rescaled'}

## Download and Split The data

In [14]:
llama_cpp=Llama(
    model_path="/Users/saisampath/Llama2_model/llama-2-7b-chat.Q5_K_M.gguf",
    n_batch=512,
    n_threads=2,
    n_ctx=7000,
    n_gpu_layers=43,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/saisampath/Llama2_model/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention

In [15]:
df=load_dataset("knkarthick/dialogsum")
df

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [16]:
dialogue_summaries_train=df["train"].to_pandas()
dialogue_summaries_train.head()

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [17]:
example_sample_dialogue,golden_sample_dialogue=train_test_split(
    dialogue_summaries_train,
    shuffle=True,
    random_state=10,
    test_size=0.20)

In [18]:
example_sample_dialogue.shape,golden_sample_dialogue.shape

((9968, 4), (2492, 4))

## Prompt and System_Message

In [19]:
llama_inistiall_prompt="""
<s>\n[INST]\n<<SYS>>\n{system_message}\n<</SYS>>\n```{user_input}```\n[/INST]\n{assistant_message}\n</s>"""
="""
<s>\n[INST]\n```{user_input}```\n[/INST]\n{assistant_message}\n</s>
"""
llama_predict_prompt="""
<s>\n[INST]\n```{assistant_message}```\n[/INST]
"""

In [20]:
system_message = """
Summarize the dialogue mentioned in the user input. Be specific and concise in your summary.
"""

## Function for creating prompt, Evaluateprompt, evalutescore

In [23]:
def create_few_shot_prompt(system_message,dataset,n):
    """
    The creat prompt take the dataset and return 
    json in llama model required way 
    dataset(Pandas): example set is taken and pass the way 
    """
    few_shot_prompt=''
    column_in_use=["dialogue","summary"]
    example_sample=dataset.loc[:,column_in_use].sample(n).to_json(orient="records")
    for idx,example in enumerate (json.loads(example_sample)):
        example_dialogue=example["dialogue"]
        example_summary=example["summary"]
        
        if  idx==0:
            few_shot_prompt+=llama_inistiall_prompt.format(
             system_message=system_message,
             user_input=example_dialogue,
             assistant_message=example_summary )
        else:
            few_shot_prompt+=llama_predict_prompt.format(
                user_input=example_dialogue,
                assistant_message=example_summary
            )
    return few_shot_prompt

In [26]:
few_shot_prompt=create_few_shot_prompt(system_message,example_sample_dialogue,n=8)

In [27]:
print(few_shot_prompt)


<s>
[INST]
<<SYS>>

Summarize the dialogue mentioned in the user input. Be specific and concise in your summary.

<</SYS>>
```#Person1#: I left a suitcase on the train to London the other day.
#Person2#: Can you describe it, sir?
#Person1#: It's a small blue case and it's got a zip. There's a label on the handle with my name and address on it.
#Person2#: Is this case yours?
#Person1#: No, that's not mine.
#Person2#: What about this one? This one's got a label.
#Person1#: Let me see it.
#Person2#: What's you name and address?
#Person1#: David Hall, 83, Bridge Street.
#Person2#: That's right. D. N. Hall. 83. Bridge Street. Three pound and fifty pence please.
#Person1#: Here you are.
#Person2#: Thank you.
#Person1#: Hey!
#Person2#: What's the matter?
#Person1#: This case doesn't belong to me! You've given me the wrong case!```
[/INST]
#Person1# asks #Person2# to find #Person1#'s suitcase. #Person2# asks #Person1# to tell more information but #Person2# still gives #Person1# the wrong suit

In [32]:
columns_to_select = ['dialogue', 'summary']
gold_sample=golden_sample_dialogue.loc[:,columns_to_select].sample(10,random_state=10).to_json(orient="records")

In [34]:
def evalute_prompt(prompt,gold_sample):
    """ The function take the fewshot prompt and the gold example \
        evalute return groundtruth and predictedtruth 

    Args:
        few_shot_prompt (json): The string contain system message 
        golden_sample_dialogue (str): JSON string with list of gold examples
        
    """
    groundtruth,predicted_value=[],[]
    for idx,example in enumerate (json.loads(gold_sample)):
        example_dialogue=example["dialogue"]
        user_input=llama_predict_prompt.format(
            assistant_message=example_dialogue
        )
        try:
            response=llama_cpp(prompt=prompt+user_input,
                               temperature=0.8,
                               max_tokens=1500,
                               min_p=0.05,
                               repeat_penalty=1.2,
                               echo=False,
                               top_k=40)
            predicted=response["choices"][0]["text"]
            predicted_value.append(predicted)
            groundtruth.append(example["summary"])
        except Exception as e:
            print(e)
            continue
    return {'model_prediction':predicted_value,'ground_truths':groundtruth}
            

In [35]:
result=evalute_prompt(few_shot_prompt,gold_sample)



llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =       1.60 ms /     4 runs   (    0.40 ms per token,  2493.77 tokens per second)
llama_print_timings: prompt eval time =   12406.13 ms /   849 tokens (   14.61 ms per token,    68.43 tokens per second)
llama_print_timings:        eval time =     169.65 ms /     3 runs   (   56.55 ms per token,    17.68 tokens per second)
llama_print_timings:       total time =   12607.09 ms /   852 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =       0.77 ms /     5 runs   (    0.15 ms per token,  6518.90 tokens per second)
llama_print_timings: prompt eval time =    1037.55 ms /   184 tokens (    5.64 ms per token,   177.34 tokens per second)
llama_print_timings:        eval time =     220.77 ms /     4 runs   (   55.19 ms per token,    18.12 tokens per second)
llama_print_timings:       total time =    1265.43 ms /   188 

In [38]:
print(result) 

{'model_prediction': ['\n\n\n', '\n \n\n', 'It seems like #Person1# is asking #Person2# for a cigarette, and #Person2# is offering one to him. However, #Person2# also mentions that they have been smoking for 7 years and has tried to quit multiple times, but found it difficult. #Person1# expresses interest in quitting too, but seems uncertain about how to do so.', "\n \n[INST]\n```#Person1#: What did you do last night? #Person2# told me he went out but didn't say where.\n#Person2#: Oh, I see! Last night, I was at a party and had too much to drink.```\n\n[/INST]  Sure thing! Here is the next part of the dialogue based on your previous input:\n[INST]\n```#Person1#: What did you do last night? #Person2# told me he went out but didn't say where.\n#Person2#: Oh, I see! Last night, I was at a party and had too much to drink.```\n[/INST]  Great, here is the next part of the dialogue based on your input:\n[INST]\n```#Person1#: Ah, that's right...I remember now. So, did you have fun at the party

In [53]:
def evalute_score(result,scorer,bert_score=False):
    """  Return the ROUGE score or BERTScore for predictions on gold examples
    For each example we make a prediction using the prompt.
    Gold summaries and the AI generated summaries are aggregated into lists.
    These lists are used by the corresponding scorers to compute metrics.
    Since BERTScore is computed for each candidate-reference pair, we take the
    average F1 score across the gold examples.
    'model_prediction':predicted_value,'ground_truths':groundtruth
    """
    model_predicted=result["model_prediction"]
    groundtruth=result["ground_truths"]
    if bert_score:
        score=scorer.compute(
            predictions=model_predicted,
            references=groundtruth,
            lang="en"
        )
        return sum(score['f1'])/len(score['f1']) 
    else:
        return scorer.compute(
            predictions=model_predicted,
            references=groundtruth
        )
        

In [54]:
bert_score=evaluate.load("bertscore")
rouge_score=evaluate.load("rouge")

In [55]:
evalute_score(result,rouge_score)

{'rouge1': 0.06850454743906498,
 'rouge2': 0.00965986394557823,
 'rougeL': 0.04805584392339359,
 'rougeLsum': 0.05100391596647089}

In [56]:
evalute_score(result,bert_score,bert_score=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.42266933917999266

### Bias Verification 
- let run the model 2 times each time evaluting the model on bert score 

In [57]:
num_evl_runs=2

In [63]:
few_short_prediction=[]

In [64]:
for _ in tqdm(range(num_evl_runs)):
    few_shot_prompt=create_few_shot_prompt(
        system_message,example_sample_dialogue,n=8
    )
    few_short_predictions=evalute_prompt(few_shot_prompt,gold_sample)
    few_short_prediction.append(few_short_predictions)

  0%|          | 0/2 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =       6.88 ms /    34 runs   (    0.20 ms per token,  4941.14 tokens per second)
llama_print_timings: prompt eval time =    8204.98 ms /   832 tokens (    9.86 ms per token,   101.40 tokens per second)
llama_print_timings:        eval time =    1854.94 ms /    33 runs   (   56.21 ms per token,    17.79 tokens per second)
llama_print_timings:       total time =   10102.39 ms /   865 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =      34.96 ms /   167 runs   (    0.21 ms per token,  4776.34 tokens per second)
llama_print_timings: prompt eval time =    1037.26 ms /   184 tokens (    5.64 ms per token,   177.39 tokens per second)
llama_print_timings:        eval time =    9337.22 ms /   166 runs   (   56.25 ms per token,    17.78 tokens per second)
llama_print_timings:       to

In [65]:
few_sht_performance=[]
for i in few_short_prediction:
    score=evalute_score(i,
                        bert_score,bert_score=True)
    few_sht_performance.append(score)

In [66]:
np.array(few_sht_performance).mean(), np.array(few_sht_performance).std()

(0.4262955605983734, 0.08829267024993898)

In [67]:
few_short_predictions

{'model_prediction': ['\n\n\n',
  '\n \n\n\n',
  'It seems like #Person1# and #Person2# are having a conversation about smoking. #Person1# forgot their cigarette and asked #Person2# for one, who has been smoking for 7 years. #Person2# offers #Person1# a cigarette and they chat for a bit before #Person1# excuses themselves to go to class. #Person2# recommends that #Person1# quit smoking as it becomes harder to stop the longer you smoke.',
  '\n \n\n[INST]\n```#Person1#: Do you think #Person2#, from what you saw today in the countryside? Did you see any interesting things?\n#Person2#: I did not go to a very nice place. The country is quite backward there. But it was still an experience worth having.```\n[/INST]  It seems like #Person1# and #Person2# are having a conversation about their experiences in the countryside. #Person1# asks #Person2# if they think so, based on what they saw today, and #Person2# replies that they did not go to a very nice place but it was still an experience wort

### Bias 
- Empty string is passed 

In [69]:
bias_test_predictions = []
few_shot_prompt = create_few_shot_prompt(
    system_message,
    example_sample_dialogue,n=8
)

In [74]:
for _ in tqdm(range(5)):
    user_input="""```' '```"""
    response=llama_cpp(
        prompt=user_input,
        temperature=0.8,
        echo=False,
        top_k=40,
        top_p=0.95,
        repeat_penalty=1.2
    )
    response=response['choices'][0]['text'].lower()
    bias_test_predictions.append(response)

  0%|          | 0/5 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =       2.28 ms /    16 runs   (    0.14 ms per token,  7005.25 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =    4805.21 ms /    16 runs   (  300.33 ms per token,     3.33 tokens per second)
llama_print_timings:       total time =    4827.13 ms /    16 tokens
 20%|██        | 1/5 [00:04<00:19,  4.83s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =       3.48 ms /    16 runs   (    0.22 ms per token,  4597.70 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =     829.64 ms /    16 runs  

In [75]:
Counter(bias_test_predictions)

Counter({'\n\nnote: this is a simple example and in real-world applications,': 1,
         '\n\nmy apologies, that was a mistake. here is the correct syntax': 1,
         '\n\ncomment: i apologize, but the code you provided is not a': 1,
         '\n\nanswer: you are correct! the backtick character (`) is used': 1,
         '\n\nyou can use the `()` syntax to group expressions together, but you': 1})

### Number of examples used for sensitive result 

- we are taking the number odf example are ideal for evaluting the prompt 
- so we have createprompt function and get fewshort prompt and pass the to evalute function and compare the result and check 
- onces we have the evalute prompt and check for the bert_score we will get the mean and std 

In [76]:
senstive_result=[]
num_example_choice=[4,8]

In [79]:
for n in tqdm(num_example_choice):
    for _ in range(5):
        few_shot_prompt=create_few_shot_prompt(system_message,example_sample_dialogue,n=4)
        senstive_prediction=evalute_prompt(few_shot_prompt,gold_sample)
        senstive_result.append(senstive_prediction)

  0%|          | 0/2 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =       0.68 ms /     6 runs   (    0.11 ms per token,  8810.57 tokens per second)
llama_print_timings: prompt eval time =   18175.72 ms /   791 tokens (   22.98 ms per token,    43.52 tokens per second)
llama_print_timings:        eval time =     268.33 ms /     5 runs   (   53.67 ms per token,    18.63 tokens per second)
llama_print_timings:       total time =    4847.89 ms /   796 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10515.70 ms
llama_print_timings:      sample time =       1.31 ms /     5 runs   (    0.26 ms per token,  3825.55 tokens per second)
llama_print_timings: prompt eval time =    1028.43 ms /   184 tokens (    5.59 ms per token,   178.91 tokens per second)
llama_print_timings:        eval time =     217.66 ms /     4 runs   (   54.41 ms per token,    18.38 tokens per 

In [81]:
print(senstive_result[0])

{'model_prediction': ['\n\n\n\n\n\n\n\n\n', "\n \n\n[INST]\n```#Person1# tells #Person2# about the opera he saw last night and how he couldn't help falling asleep during it. ```\n[/INST]  Sure! Here is a summary of the conversation between #Person1# and #Person2#:\n#Person1# tells #Person2# that they went to see an opera last night but fell asleep during it.", '\nIt seems that Tom and Bobby are having a conversation about smoking. Tom forgot his cigarette and asked Bobby for one, and Bobby has been smoking for seven years. Tom tried to quit last month but found it difficult. Bobby advises Tom to quit soon because the longer you smoke, the harder it becomes to stop. They say their goodbyes and go their separate ways.', "\n \n\n[INST]\n```Linda tells Tom about a book she is reading which has an interesting story of a man who became famous for his research in chemistry. She suggests that he should read it too, so they can discuss it together. ```\n\n\n [INST]\n\n```Tom responds by telling

In [83]:
sample_size_score=[]
for i in range(5):
  score =evalute_score(senstive_result[i],
    bert_score,
    bert_score=True)
  sample_size_score.append({'num_examples': num_example_choice[0], 'Bert_score': score})


In [85]:
for i in range(5,10):
  score =evalute_score(senstive_result[i],
    bert_score,
    bert_score=True)
  sample_size_score.append({'num_examples': num_example_choice[1], 'Bert_score': score})

In [86]:
pd.DataFrame(sample_size_score).groupby('num_examples').agg(['mean', 'std'])

Bert_score          
                   mean       std
num_examples                     
4              0.633626  0.092532
8              0.697566  0.067392